<a href="https://colab.research.google.com/github/Abdullah-Mehboob1/Abdullah-Mehboob1/blob/main/bestcode%20for%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

In [4]:
# Load the dataset
dataset_path = '/content/drive/MyDrive/dataset'
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')
image_size = (256, 256)
batch_size = 32
seq_length = 10

In [5]:
# Data augmentation for training and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create train and test generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important for consistent evaluation
)

Found 804 images belonging to 4 classes.
Found 540 images belonging to 4 classes.


In [6]:
# Calculate class weights
labels = train_generator.classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights_dict = dict(enumerate(class_weights))

In [7]:
# Custom CNN and ANN combined model
model = keras.Sequential([
    # CNN layers
    keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(16, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),

    # ANN layers
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# Learning rate scheduler
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7
)

In [ ]:
# Train the model with cross-validation using KFold
from sklearn.model_selection import KFold
kf = KFold(n_splits=2, shuffle=True, random_state=42)

# Store scores for each fold
accuracies = []
f1_scores = []

for train_index, val_index in kf.split(train_generator.classes):
    model.fit(
        train_generator,
        epochs=40,
        validation_data=test_generator,
        class_weight=class_weights_dict,
        callbacks=[lr_scheduler]
    )

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


51/51 ━━━━━━━━━━━━━━━━━━━━ 559s 10s/step - accuracy: 0.3377 - loss: 1.3085 - val_accuracy: 0.7759 - val_loss: 0.4094 - learning_rate: 0.0010
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 353s 7s/step - accuracy: 0.6972 - loss: 0.5888 - val_accuracy: 0.7704 - val_loss: 0.3784 - learning_rate: 0.0010
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 370s 7s/step - accuracy: 0.7273 - loss: 0.4653 - val_accuracy: 0.8241 - val_loss: 0.3230 - learning_rate: 0.0010
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 384s 7s/step - accuracy: 0.7431 - loss: 0.4324 - val_accuracy: 0.7444 - val_loss: 0.3857 - learning_rate: 0.0010
Epoch 5/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 364s 7s/step - accuracy: 0.7613 - loss: 0.3872 - val_accuracy: 0.7463 - val_loss: 0.4941 - learning_rate: 0.0010
Epoch 6/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 335s 7s/step - accuracy: 0.7238 - loss: 0.5739 - val_accuracy: 0.7481 - val_loss: 0.3501 - learning_rate: 0.0010
Epoch 7/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 337s 7s/step - accuracy: 0.7681 - loss: 0.3723 - val_accuracy: 0.7

In [1]:
 # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(test_generator)
    accuracies.append(test_acc)

    # Predict on the test set and calculate F1-score
    y_pred = model.predict(test_generator)
    y_pred_class = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes
    f1 = f1_score(y_true, y_pred_class, average="macro")
    f1_scores.append(f1)

    print(f'Test accuracy: {test_acc:.2f}')
    print(f'Test F1-score: {f1:.2f}')

print(f'Average Test accuracy: {np.mean(accuracies):.2f}')
print(f'Average Test F1-score: {np.mean(f1_scores):.2f}')

# Adjust thresholds for better precision-recall tradeoff
thresholds = np.linspace(0, 1, 100)
precisions = []
recalls = []
f1_scores = []

for threshold in thresholds:
    y_pred_adj = (y_pred > threshold).astype(int)
    precision = precision_score(y_true, y_pred_adj, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred_adj, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred_adj, average="macro", zero_division=0)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Plot Precision-Recall curve
plt.figure()
plt.plot(recalls, precisions, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

# Report optimal threshold
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f'Optimal Threshold: {optimal_threshold:.2f}')

# Recalculate metrics at optimal threshold
y_pred_optimal = (y_pred > optimal_threshold).astype(int)
optimal_f1 = f1_score(y_true, y_pred_optimal, average="macro", zero_division=0)
optimal_precision = precision_score(y_true, y_pred_optimal, average="macro", zero_division=0)
optimal_recall = recall_score(y_true, y_pred_optimal, average="macro", zero_division=0)

print(f'Optimal F1-score: {optimal_f1:.2f}')
print(f'Optimal Precision: {optimal_precision:.2f}')
print(f'Optimal Recall: {optimal_recall:.2f}')

IndentationError: unexpected indent (<ipython-input-1-d6143a4be62b>, line 2)

In [ ]:
from sklearn.metrics import precision_recall_curve

# Compute Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_true, y_pred_class, pos_label=1)

# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred_class)

# Plot confusion matrix as heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=np.unique(y_true), yticklabels=np.unique(y_true))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix Heatmap')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true and y_pred_class are already defined
# y_true: True labels
# y_pred_class: Predicted labels

# Compute metrics
precision = precision_score(y_true, y_pred_class, average=None, zero_division=0)
recall = recall_score(y_true, y_pred_class, average=None, zero_division=0)
f1 = f1_score(y_true, y_pred_class, average=None, zero_division=0)

# Compute average metrics across all classes for error bars
mean_precision = np.mean(precision)
mean_recall = np.mean(recall)
mean_f1 = np.mean(f1)

# Standard deviation for error bars
std_precision = np.std(precision)
std_recall = np.std(recall)
std_f1 = np.std(f1)

# Plotting
labels = ['Precision', 'Recall', 'F1 Score']
means = [mean_precision, mean_recall, mean_f1]
std_devs = [std_precision, std_recall, std_f1]

fig, ax = plt.subplots()
bars = ax.bar(labels, means, yerr=std_devs, capsize=5, color=['blue', 'orange', 'green'])
ax.set_ylabel('Score')
ax.set_title('Average Precision, Recall, and F1 Score with Error Bars')

# Add value labels on bars
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred_class, pos_label=1)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Sample data for illustration
epochs = list(range(1, 61))  # Example: 60 epochs
precision_history = [0.8 + 0.001*i for i in range(60)]  # Example precision values
recall_history = [0.75 + 0.0015*i for i in range(60)]  # Example recall values
f1_history = [2 * (p * r) / (p + r) for p, r in zip(precision_history, recall_history)]  # Example F1 scores
tp_history = [50 + i for i in range(60)]  # Example true positive values
fp_history = [10 + i for i in range(60)]  # Example false positive values

# Check if lists are not empty and have the same length
if (precision_history and recall_history and f1_history and tp_history and fp_history and
    len(precision_history) == len(recall_history) == len(f1_history) == len(tp_history) == len(fp_history)):

    plt.figure(figsize=(12, 6))

    # Plot precision, recall, and F1-score
    plt.subplot(1, 2, 1)
    plt.plot(epochs, precision_history, label='Validation Precision', marker='o')
    plt.plot(epochs, recall_history, label='Validation Recall', marker='o')
    plt.plot(epochs, f1_history, label='Validation F1-score', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Metrics')
    plt.legend()
    plt.title('Precision, Recall, F1-score')

    # Plot true positive and false positive
    plt.subplot(1, 2, 2)
    plt.plot(epochs, tp_history, label='True Positive', color='green', marker='o')
    plt.plot(epochs, fp_history, label='False Positive', color='red', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Counts')
    plt.legend()
    plt.title('True Positive and False Positive')

    plt.tight_layout()
    plt.show()
else:
    print("Data lists are empty or have mismatched lengths.")


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
dataset_path = '/content/drive/MyDrive/dataset'
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')
image_size = (256, 256)
batch_size = 16
seq_length = 10

# Data augmentation for training and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create train and test generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important for consistent evaluation
)

# Calculate class weights
labels = train_generator.classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights_dict = dict(enumerate(class_weights))

# Custom CNN and ANN combined model
model = keras.Sequential([
    # CNN layers
    keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(16, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),

    # ANN layers
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Learning rate scheduler
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7
)

# Directory where the checkpoints will be saved
# Directory where the checkpoints will be saved
checkpoint_path = "/content/drive/MyDrive/checkpoints/cp-{epoch:04d}.ckpt.weights.h5"  # Add .weights.h5
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq='epoch'  # Save every epoch
)

# Load the latest checkpoint if it exists
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    model.load_weights(latest_checkpoint)
    print(f"Loaded model from checkpoint: {latest_checkpoint}")
    # Extract the last epoch number from the checkpoint filename
    last_epoch = int(latest_checkpoint.split('-')[-1].split('.')[0])
else:
    print("No checkpoint found, training from scratch.")
    last_epoch = 0  # Start from the beginning

# Train the model with cross-validation using KFold
kf = KFold(n_splits=2, shuffle=True, random_state=42)

# Store scores for each fold
accuracies = []
f1_scores = []

for train_index, val_index in kf.split(train_generator.classes):
    model.fit(
        train_generator,
        epochs=50,  # Set the total number of epochs
        initial_epoch=last_epoch,  # Start from the last epoch if checkpoint exists
        validation_data=test_generator,
        class_weight=class_weights_dict,
        callbacks=[lr_scheduler, cp_callback]  # Add the checkpoint callback here
    )


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 804 images belonging to 4 classes.
Found 540 images belonging to 4 classes.
No checkpoint found, training from scratch.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3433 - loss: 1.3029
Epoch 1: saving model to /content/drive/MyDrive/checkpoints/cp-0001.ckpt.weights.h5
51/51 ━━━━━━━━━━━━━━━━━━━━ 475s 9s/step - accuracy: 0.3454 - loss: 1.3001 - val_accuracy: 0.7019 - val_loss: 0.5833 - learning_rate: 0.0010
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6442 - loss: 0.6629
Epoch 2: saving model to /content/drive/MyDrive/checkpoints/cp-0002.ckpt.weights.h5
51/51 ━━━━━━━━━━━━━━━━━━━━ 349s 7s/step - accuracy: 0.6452 - loss: 0.6611 - val_accuracy: 0.7463 - val_loss: 0.3517 - learning_rate: 0.0010
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7457 - loss: 0.4337
Epoch 3: saving model to /content/drive/MyDrive/checkpoints/cp-0003.ckpt.weights.h5
51/51 ━━━━━━━━━━━━━━━━━━━━ 340s 7s/step - accuracy: 0.7457 - loss: 0.4336 - val_accuracy: 0.7741 - val_loss: 0.3564 - learning_rate: 0.0010
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7623 - loss: 0.4046
Epoch 4

In [15]:
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import numpy as np
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define paths and parameters
dataset_path = '/content/drive/MyDrive/dataset'
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')
image_size = (256, 256)
batch_size = 16
checkpoint_dir = "/content/drive/MyDrive/checkpoints"

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Calculate class weights
labels = train_generator.classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights_dict = dict(enumerate(class_weights))

# Define model
model = keras.Sequential([
    keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(16, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Learning rate scheduler
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7
)

# Define checkpoint callback with correct filepath
checkpoint_path = os.path.join(checkpoint_dir, "cp-{epoch:04d}.weights.h5")
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq='epoch'
)

# List all checkpoint files for debugging
checkpoint_files = os.listdir(checkpoint_dir)
print("Checkpoint files:", checkpoint_files)

import glob

# List all files in the checkpoint directory
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, '*.weights.h5'))
print("Checkpoint files:", checkpoint_files)

# Find the latest checkpoint file by sorting the list
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    print(f"Latest checkpoint path: {latest_checkpoint}")
else:
    latest_checkpoint = None
    print("No checkpoint files found.")

# Train the model with the checkpoint callback
model.fit(
    train_generator,
    epochs=50,  # Set the total number of epochs
    initial_epoch=last_epoch,  # Start from the last epoch if checkpoint exists
    validation_data=test_generator,
    class_weight=class_weights_dict,
    callbacks=[lr_scheduler, cp_callback]
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 804 images belonging to 4 classes.
Found 540 images belonging to 4 classes.
Checkpoint files: ['cp-0010.ckpt.weights.h5', 'cp-0011.ckpt.weights.h5', 'cp-0012.ckpt.weights.h5', 'cp-0013.ckpt.weights.h5', 'cp-0014.ckpt.weights.h5', 'cp-0015.ckpt.weights.h5', 'cp-0016.ckpt.weights.h5', 'cp-0017.ckpt.weights.h5', 'cp-0018.ckpt.weights.h5', 'cp-0019.ckpt.weights.h5', 'cp-0020.ckpt.weights.h5', 'cp-0021.ckpt.weights.h5', 'cp-0022.ckpt.weights.h5', 'cp-0023.ckpt.weights.h5', 'cp-0024.ckpt.weights.h5', 'cp-0025.ckpt.weights.h5', 'cp-0026.ckpt.weights.h5', 'cp-0027.ckpt.weights.h5', 'cp-0028.ckpt.weights.h5', 'cp-0029.ckpt.weights.h5', 'cp-0030.ckpt.weights.h5', 'cp-0031.ckpt.weights.h5', 'cp-0032.ckpt.weights.h5', 'cp-0033.ckpt.weights.h5', 'cp-0034.ckpt.weights.h5', 'cp-0035.ckpt.weights.h5', 'cp-0036.ckpt.weights.h5', 'cp-0037.ckpt.weights.h5', 'cp-0038.ckpt.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/51 ━━━━━━━━━━━━━━━━━━━━ 2:18 6s/step - accuracy: 0.2694 - loss: 1.3621

KeyboardInterrupt: 

In [7]:
 # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(test_generator)
    accuracies.append(test_acc)

    # Predict on the test set and calculate F1-score
    y_pred = model.predict(test_generator)
    y_pred_class = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes
    f1 = f1_score(y_true, y_pred_class, average="macro")
    f1_scores.append(f1)

    print(f'Test accuracy: {test_acc:.2f}')
    print(f'Test F1-score: {f1:.2f}')

print(f'Average Test accuracy: {np.mean(accuracies):.2f}')
print(f'Average Test F1-score: {np.mean(f1_scores):.2f}')

# Adjust thresholds for better precision-recall tradeoff
thresholds = np.linspace(0, 1, 100)
precisions = []
recalls = []
f1_scores = []

for threshold in thresholds:
    y_pred_adj = (y_pred > threshold).astype(int)
    precision = precision_score(y_true, y_pred_adj, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred_adj, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred_adj, average="macro", zero_division=0)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Plot Precision-Recall curve
plt.figure()
plt.plot(recalls, precisions, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

# Report optimal threshold
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f'Optimal Threshold: {optimal_threshold:.2f}')

# Recalculate metrics at optimal threshold
y_pred_optimal = (y_pred > optimal_threshold).astype(int)
optimal_f1 = f1_score(y_true, y_pred_optimal, average="macro", zero_division=0)
optimal_precision = precision_score(y_true, y_pred_optimal, average="macro", zero_division=0)
optimal_recall = recall_score(y_true, y_pred_optimal, average="macro", zero_division=0)

print(f'Optimal F1-score: {optimal_f1:.2f}')
print(f'Optimal Precision: {optimal_precision:.2f}')
print(f'Optimal Recall: {optimal_recall:.2f}')

IndentationError: unexpected indent (<ipython-input-7-d6143a4be62b>, line 2)

In [ ]:
from sklearn.metrics import precision_recall_curve

# Compute Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_true, y_pred_class, pos_label=1)

# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred_class)

# Plot confusion matrix as heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=np.unique(y_true), yticklabels=np.unique(y_true))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix Heatmap')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true and y_pred_class are already defined
# y_true: True labels
# y_pred_class: Predicted labels

# Compute metrics
precision = precision_score(y_true, y_pred_class, average=None, zero_division=0)
recall = recall_score(y_true, y_pred_class, average=None, zero_division=0)
f1 = f1_score(y_true, y_pred_class, average=None, zero_division=0)

# Compute average metrics across all classes for error bars
mean_precision = np.mean(precision)
mean_recall = np.mean(recall)
mean_f1 = np.mean(f1)

# Standard deviation for error bars
std_precision = np.std(precision)
std_recall = np.std(recall)
std_f1 = np.std(f1)

# Plotting
labels = ['Precision', 'Recall', 'F1 Score']
means = [mean_precision, mean_recall, mean_f1]
std_devs = [std_precision, std_recall, std_f1]

fig, ax = plt.subplots()
bars = ax.bar(labels, means, yerr=std_devs, capsize=5, color=['blue', 'orange', 'green'])
ax.set_ylabel('Score')
ax.set_title('Average Precision, Recall, and F1 Score with Error Bars')

# Add value labels on bars
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred_class, pos_label=1)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Sample data for illustration
epochs = list(range(1, 61))  # Example: 60 epochs
precision_history = [0.8 + 0.001*i for i in range(60)]  # Example precision values
recall_history = [0.75 + 0.0015*i for i in range(60)]  # Example recall values
f1_history = [2 * (p * r) / (p + r) for p, r in zip(precision_history, recall_history)]  # Example F1 scores
tp_history = [50 + i for i in range(60)]  # Example true positive values
fp_history = [10 + i for i in range(60)]  # Example false positive values

# Check if lists are not empty and have the same length
if (precision_history and recall_history and f1_history and tp_history and fp_history and
    len(precision_history) == len(recall_history) == len(f1_history) == len(tp_history) == len(fp_history)):

    plt.figure(figsize=(12, 6))

    # Plot precision, recall, and F1-score
    plt.subplot(1, 2, 1)
    plt.plot(epochs, precision_history, label='Validation Precision', marker='o')
    plt.plot(epochs, recall_history, label='Validation Recall', marker='o')
    plt.plot(epochs, f1_history, label='Validation F1-score', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Metrics')
    plt.legend()
    plt.title('Precision, Recall, F1-score')

    # Plot true positive and false positive
    plt.subplot(1, 2, 2)
    plt.plot(epochs, tp_history, label='True Positive', color='green', marker='o')
    plt.plot(epochs, fp_history, label='False Positive', color='red', marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Counts')
    plt.legend()
    plt.title('True Positive and False Positive')

    plt.tight_layout()
    plt.show()
else:
    print("Data lists are empty or have mismatched lengths.")
